<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Infeniería de feature

Crear un modelo base para analizar el dataset de partidos<br>
v1.1

### Objetivos: Generar un modelo base sin utilizar Inteligencia Artificial para predecir el resultado de un partido de Futbol.

*   Implemetar el código de descarga de un archivo csv.
*   Comprender la lectura de un archivo csv con Pandas.
*   Realizar la descripción estadística con Pandas.
*   Identificar y eliminar datos faltantes.
*   Explorar los datos y aplicar filtros para alquileres en ARS.
*   Representar graficamente los datos para observar la tendencia.
*   Definir los valores de X, y como variables necesarias para un modelo.
*   Definir el criterio para dividir el dataset en entrenamiento (train) y evaluación (test).
* Crear un modelo base simple para calcular el precio promedio de un alquiler
*  Implementar métricas.

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('partidos.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv > partidos.csv
    else:
        !wget partidos.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.

df = pd.read_csv("partidos.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas del DataFrame df
df.head()

In [ ]:
# ¿Cuántos datos quedaron para analizar?
# (filas, columnas)
df.shape

## Fin de la limpieza
No hay datos incompletos o mal ingresados en el dataset, está limpio! No es necesario ningún tipo de limpieza

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Muestra las 5 primeras filas del DataFrame df
df.head()

Se puede observar que está el puntaje de cada partido pero no el resultado final (wins, loses, draws) --> (ganó, perdió, empató). Agregaremos una columna "results" con el resultado por partido.

In [ ]:
# Haciendo copia de df
df1 = df.copy()

In [ ]:
# Cambiar los nombres de las columnas.
# Para ello, del DataFrame df accedemos a columns que contiene todos los nombres de las columnas.
# Las cuales se van a sobreescribir con los nuevos nombres.
df1.columns = ['fecha', 'equipo_local', 'equipo_visitante', 'resultado_local', 'resultado_visitante', 'torneo', 'ciudad', 'pais', 'neutral']
df1.head()

In [ ]:
# Sa filtra el dataset para agregar una nueva columna llamada "resultados".
# Esta columna permite incrementar la información a partir de la ya disponible.
# Para ello, la columna resultados se completará con el retorno de una lambda.
# La lambda va a recorrer todo el DataFrame para verificar los condicionales establecidos:
# 'ganó' if x.resultado_local > x.resultado_visitante
#'perdió' if x.resultado_local < x.resultado_visitante
# else 'empató'
# El axis=1, es para que se aplica la lambda para cada fila del DataFrame.
df1['resultados'] = df1.apply(lambda x : 'ganó' if x.resultado_local > x.resultado_visitante else 'perdió' if x.resultado_local < x.resultado_visitante else 'empató', axis=1)
df1.head()

In [ ]:
# En el registro de la totalidad de partidos, ¿Cuántos partidos se ganaron, cuántos se pierden y cuántos se empataron?
df1['resultados'].value_counts()

In [ ]:
# En la columna equipo_local, se cuenta cuántos valores únicos se encuentran.
print("¿Cuántos países hay registrados?:", df1['equipo_local'].nunique())

In [ ]:
# Partidos jugados por Argentina de local:
pais_analizar = 'Argentina'

# Se filtra df1 para extraer la información del equipo Argentino.
df_arg = df1[ df1['equipo_local'] == pais_analizar]

# countplot, similar a un grágico de barrar, permite comparar la frecuencia de los datos a estudiar,
# en este caso, la freciencia de los resultados de los partidos.
# sns, alias de Seaborn
# x='resultados', nombre de la columna que se desea observar la frecuencia
# data=df_arg, el DataFrame von la información.
# set_title(f"Histórico de {pais_analizar} permite asociar un título al gráfico
# plt.show() muestra el gráfico.
sns.countplot(x='resultados', data=df_arg).set_title(f"Histórico de {pais_analizar}")
plt.show()

# El countplot nos resuelve no tener que hacer lo siguiente:
#sns.barplot(x=df_arg['result'].value_counts().index, y=df_arg['result'].value_counts())

In [ ]:
# Simplificar el resultado del datset, solo se tendrá encuenta si el local gana (target=1)
# Caso contrario target = 0
df1['target'] = df1['resultados'].apply(lambda x: 1 if x == 'ganó' else 0)
df1

# Entrenar modelo
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle"></div>

In [ ]:
# Crear un numpy array "X" de los features de entrada que se utilizarian
# para entrar al modelo
# Se ha seleccionado los nombres de los equipos rivales, el consejo que
# que para aumentar la performance en vez de ingresar el nombre se realice
# binaryEncoding
# (queda para el almno que desee realizar el proyecto de este dataset)
# doble corchetes porque se accede a los valores de dos columnas.
X = df1[['equipo_local', 'equipo_visitante']].values

# Crear un numpy array "y" con la salida/objetivo --> target
y = df1['target'].values

In [ ]:
# Crear los dataset de entrenamiento (train) y evaluación (test)
# utilizando la herramienta train_test_split de scikit-learn,
# con la proporción 70%30%
# Ojo! Tener en cuenta que los dataset ahora son numpy array
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Modelo base
# np, NumPy
# método de NumPy random.randint, devuelve un número aleatorio entre 0 y 2 (El 2 no está incluido)
def predict(X):
    return np.random.randint(0, 2, size=X.shape[0])

In [ ]:
# Obtener la salida según el modelo base
y_hat_base = predict(X_test)

In [ ]:
# Dibujar la matriz de confusión con y_test e y_hat_base
# Matriz de Confusion
# Se utliza la matriz de confusión para evaluar la precisión de una clasificación.
# cm = confusion_matrix(), necesita dos variables que contengan los valores a comparar.

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat_base)

# Código para relizar la representación gráfica con los resultados
# Se crea la varible cmd, que almacena visualization de la Confusion Matrix 
# Necesita la variable cm que contiene los resultados de la comparación entre los valores reales y predicción
# display_labels, se especifica las etiquetas de las categorias que se evalúan.
cmd = ConfusionMatrixDisplay(cm, display_labels=['perdió','ganó'])

# Con cmd.plot se especifica el mapa de colores reconocido por matplotlib.
cmd.plot(cmap=plt.cm.Oranges)

# Para mostrar la figura
plt.show()

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat_base, normalize=True)

In [ ]:
# Calcular el f1_score
from sklearn.metrics import f1_score
f1_score(y_test, y_hat_base)